Skolemization-Related Explorations
==================

This is a temporary notebook for exploring issues related to a skolemize() method for KnownTruth objects, in particular for known truths involving universal ($\forall$) and existential ($\exists$) quantifiers.

The notebook was generated initially as a copy of the Forall tutorial notebook, and thus while under construction many of the cells and other elements of the original notebook remain.

($\forall$) Universal quantification is another core concept in **Prove-It**.  A `Forall` operation, formatted with the $\forall$ symbol, is used to represent universal quantification.  For example, $\forall_x P(x)$ means that $P(x)$ is true for any instance of $x$.  $P(x)$ holds true universally over instances of $x$.  Like `Implies`, `Forall` is a core concept but is defined outside of the core in the `proveit.logic` package. It is known in the core for use in the *specialization* and *generalization* derivation steps discussed below.

First, let us import some necessary information then consider an example of a `Forall` object like $\forall_{x \,\in\, S \,|\, Q(x), R(x)} P(x)$:

In [1]:
from proveit import Function, ExprList, Lambda, Literal
from proveit.logic import Forall, Exists, InSet, Equals
from proveit.number import Less, Add, Frac
from proveit._common_ import a, b, x, y, z, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, T
%begin skolemization_explorations

In [2]:
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)

basicForallExpr: forall_{x in S | Q(x) , R(x)} P(x)

In [3]:
Qa = Function(Q, a)
Ra = Function(R, a)
Lc = Literal('c')
basicForallExpr.specialize({x:a}, assumptions=(basicForallExpr, InSet(a, S), Qa, Ra))

{forall_{x in S | Q(x) , R(x)} P(x) , a in S , Q(a) , R(a)} |- P(a)

In [4]:
Qa = Function(Q, a)
Ra = Function(R, a)
Rb = Function(R, b)
Lc = Literal('c')
QLc = Function(Q, Lc)
RLc = Function(R, Lc)
basicForallExprSpec = basicForallExpr.specialize({x:Lc}, assumptions=(basicForallExpr, InSet(Lc, S), QLc, RLc))

basicForallExprSpec: {forall_{x in S | Q(x) , R(x)} P(x) , c in S , Q(c) , R(c)} |- P(c)

In [5]:
def testFxn():
    import pdb
    pdb.set_trace()
    return basicForallExpr.usedVars()

In [6]:
basicForallExprSpec.usedVars()

{P}

In [7]:
basicForallExprSpec.usedLiterals()

{c}

In [8]:
(list(basicForallExprSpec.subExprIter()))

[P, c]

In [114]:
list(((list(basicForallExprSpec.subExprIter()))[1]).subExprIter())

[]

In [10]:
basicThereExistsExpr = Exists(x, Px, conditions=[Qx], domain=S)

basicThereExistsExpr: exists_{x in S | Q(x)} P(x)

In [11]:
basicThereExistsExpr

exists_{x in S | Q(x)} P(x)

In [12]:
forallThereExistsExpr = Forall(
    x,
    Exists(
        {a, b},
        Equals(x, Frac(a, b)),
        conditions={Ra, Rb}
    ),
    conditions={Qx}
)
forallThereExistsExpr

forall_{x | Q(x)} [exists_{b, a | R(b) , R(a)} (x = (a / b))]

In [13]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "from proveit import Function, ExprList, Lambda, Literal\nfrom proveit.logic import Forall, Exists, InSet, Equals\nfrom proveit.number import Less, Add, Frac\nfrom proveit._common_ import a, b, x, y, z, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, T\nget_ipython().run_line_magic('begin', 'skolemization_explorations')",
  "basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)\nproveit.magics.Assignments(['basicForallExpr'], [basicForallExpr])",
  "Qa = Function(Q, a)\nRa = Function(R, a)\nLc = Literal('c')\nbasicForallExpr.specialize({x:a}, assumptions=(basicForallExpr, InSet(a, S), Qa, Ra))",
  "Qa = Function(Q, a)\nRa = Function(R, a)\nRb = Function(R, b)\nLc = Literal('c')\nQLc = Function(Q, Lc)\nRLc = Function(R

In [14]:
from proveit import Axiom, Context
basicThereExistsExprAxiom = Axiom(basicThereExistsExpr, Context(), 'basicThereExistsExprAxiom')

basicThereExistsExprAxiom: 	step type	requirements	statement
0	axiom		|- exists_{x in S | Q(x)} P(x)
	tutorial.basicThereExistsExprAxiom

In [15]:
# dir()

In [16]:
# globals()

In [17]:
# forallThereExistsExpr.specialize({x:x}, assumptions={forallThereExistsExpr, Qx})

We can use the `exprInfo()` method to examine the internal structure of the expression:

In [18]:
basicForallExpr.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 13body: 3conditions: 4,
3,Operation,operator: 5operand: 13,
4,ExprList,"6, 7, 8",
5,Variable,,
6,Operation,operator: 9operands: 10,
7,Operation,operator: 11operand: 13,
8,Operation,operator: 12operand: 13,
9,Literal,,


`Forall` derives from `OperationOverInstances` (`proveit._core_.expression.operation.operation_over_instances.OperationOverInstances` aliased as `proveit.OperationOverInstances`) which generally defines an operation that acts on a **lambda** map with optional conditions.  The idea is like a "functional" (a function of a function).  It operates over the range of instances for the **lambda** parameters for which the condition is satisfied.  Other examples of **expression** types that derive from `OperationOverInstances` are $\exists$, $\sum$, and $\prod$.  

In our example above, we see that the domain $S$ is internally represented via the first condition of the conditional **lambda** (see Line 2 of the `exprInfo()` output).  In the external representation, it is displayed more compactly along with the introduction of $x$ before the vertical line that precedes the other conditions.  This is a matter of presentation style that is independent of how **Prove-It** treats this expression.  As far as **Prove-It** is concerned, $x \in S$ is simply a condition no different from $Q(x)$ and $R(x)$.

We can view a formatted version of the original object like this:

In [19]:
basicForallExpr

forall_{x in S | Q(x) , R(x)} P(x)

We can use Python's `dir()` function to get a list of our `Forall` object's attributes and methods:

In [20]:
dir(basicForallExpr)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allConditions',
 '_allDomains',
 '_allInstanceVars',
 '_checkRelabelMap',
 '_class_path',
 '_clear_',
 '_config_latex_tool',
 '_coreInfo',
 '_createOperand',
 '_expandingIterRanges',
 '_extractCoreInfo',
 '_extractExprClass',
 '_extractInitArgs',
 '_extractMyInitArgs',
 '_extractReferencedObjIds',
 '_extractStyle',
 '_formatMultiOperator',
 '_formatted',
 '_generate_unique_rep',
 '_genericExpr',
 '_implicitOperator',
 '_init_argname_mapping_',
 '_make',
 '_meaningData',
 '_meaning_id',
 '_operator_',
 '_repr_html_',
 '_requirements',
 '_restrictionChecked',
 '_setContext',
 '_specializeUnravelingTheorem',
 '_styleData',

and the general Python-provided object attribute `__dict__` (using two underscores on each end) to view all the attribute names and values for our specific `Forall` object:

In [21]:
basicForallExpr.__dict__

{'operator_or_operators': forall,
 'operand_or_operands': x -> P(x) | x in S , Q(x) , R(x),
 'operator': forall,
 'operators': (forall),
 'operand': x -> P(x) | x in S , Q(x) , R(x),
 'operands': (x -> P(x) | x in S , Q(x) , R(x)),
 '_subExpressions': (forall, x -> P(x) | x in S , Q(x) , R(x)),
 '_genericExpr': forall_{_x_ in S | Q(_x_) , R(_x_)} P(_x_),
 '_meaningData': <proveit._core_._unique_data._MeaningData at 0x105a0e048>,
 '_styleData': <proveit._core_._unique_data._StyleData at 0x105a0e2e8>,
 '_meaning_id': 1873434364747766254,
 '_coreInfo': ('Operation',),
 '_requirements': (),
 '_style_id': 5946875236989378449,
 'instanceExpr': P(x),
 'instanceVar': x,
 'domain': S,
 'conditions': (x in S , Q(x) , R(x))}

Some of the various parts of the `Forall` **expression** may be accessed as follows:

In [22]:
basicForallExpr.instanceVar        # Variable whose value defines the instance.
                                   # This attribute returns just the first instance variable
                                   # if there is more than one.

x

In [23]:
basicForallExpr.allInstanceVars()  # The list of all variables whose values define the instance
                                   # (may be one or more variables).

[x]

In [24]:
basicForallExpr.instanceExpr       # The expression being quantified over.

P(x)

In [25]:
basicForallExpr.conditions         # The list of conditions of the universal quantification.

(x in S , Q(x) , R(x))

In [26]:
basicForallExpr.domain             # Domain of the instance variable.
                                   # If there are multiple instance variables each with its own domain,
                                   # this attribute returns just the domain for the 1st variable.

S

In [27]:
basicForallExpr.allDomains()       # The set of all domains for all instance variables
                                   # (might be 0, 1, or more in the list)

[S]

In [28]:
basicForallExpr.explicitConditions()  # Returns the list of conditions that appear after the
                                      # vertical line in the notation (i.e., exluding the domain condition(s)).

[Q(x), R(x)]

### A More Complex Forall Example

We can construct more complex `Forall` **expression**s involving multiple instance-defining variables, each from the same domain, like this:

In [29]:
secondForallExpr = Forall([x, y], Pxy, conditions=[Qx, Ry], domain = S)

secondForallExpr: forall_{x, y in S | Q(x) , R(y)} P(x , y)

or specify individual domains for the instance variables:

In [30]:
thirdForallExpr = Forall([x, y], Pxy, conditions=[Qx, Ry], domains = [S, T])

thirdForallExpr: forall_{(x, y) in S * T | Q(x) , R(y)} P(x , y)

The various parts of such `Forall` **expression**s may be accessed as before, with a few special circumstances:

In [31]:
thirdForallExpr.instanceVar              # Obtain the first of the instance-defining variables

x

In [32]:
thirdForallExpr.allInstanceVars()        # Obtain a list of ALL instance-defining variables

[x, y]

In [33]:
thirdForallExpr.allInstanceVars()[-1]    # Obtain the LAST of all instance-defining variables

y

In [34]:
thirdForallExpr.instanceExpr             # Obtain the first nested expression being quantified over

forall_{y in T | R(y)} P(x , y)

In [35]:
thirdForallExpr.explicitInstanceExpr()   # Obtain the explicit final expression being quantified over

P(x , y)

In [36]:
thirdForallExpr.domain                   # Obtain the domain if just a single domain, or the domain of
                                         # the 1st instance-defining variable if multiple domains

S

In [37]:
thirdForallExpr.allDomains()             # Obtain a list of all domains

[S, T]

In [38]:
thirdForallExpr.conditions               # Obtain conditions for first instance-defining variable

[x in S, Q(x)]

In [39]:
thirdForallExpr.allConditions()          # Obtain conditions for all instance-defining variables

[x in S, Q(x), y in T, R(y)]

In [40]:
thirdForallExpr.instanceExpr.conditions  # Obtain conditions for 2nd (or 1st-nested) instance-defining variable

(y in T , R(y))

Specialization (or Universal Instantiation)
======

The *specialization* derivation (or universal instantiation) step uses *expression substitution* internally.  The difference is that *specialization* has proof implications and enforces the extra restrictions to justify these proof implications.  It also eliminates one or more of the outer $\forall$ operations.


### Basic Specialization

Let us take our basic/generic example of the `Forall` expression above and specialize it with a particular "instance" expression.  To do so, we will make assumptions to trivially allow this derivation step to be taken (just to show how this works).

In [41]:
from proveit import Function, ExprList
from proveit._common_ import fy
from proveit.logic import InSet
assumptions = ExprList(basicForallExpr, InSet(fy, S), Function(Q, fy), Function(R, fy))

assumptions: (forall_{x in S | Q(x) , R(x)} P(x) , f(y) in S , Q(f(y)) , R(f(y)))

`InSet()` is another core concept that is defined outside of the core in `proveit.logic`.  It represents the set membership operation using the $\in$ symbol.  It is needed as a core concept specifically for the purpose of ensuring that universal quantification requirements are met (the "instance" expression must be "in" the domain set).

To implement the specialization or universal instantiation where the instance variable $x$ is replaced with the specific instance or value $f(y)$, we have:

In [42]:
basicForallSpec = basicForallExpr.specialize({x:fy}, assumptions=assumptions)
basicForallSpec

{forall_{x in S | Q(x) , R(x)} P(x) , f(y) in S , Q(f(y)) , R(f(y))} |- P(f(y))

We have proven, somewhat trivially, that $P(f(y))$ is true assuming that $\forall_{x \in S~|~Q(x)} P(x)$, $f(y) \in S$, $Q(f(y))$ are all true statements.

Let us take a look at the proof for this statement:

In [43]:
basicForallSpec.proof()

step type	requirements	statement
0	specialization	1, 2, 3, 4	{forall_{x in S | Q(x) , R(x)} P(x) , f(y) in S , Q(f(y)) , R(f(y))} |- P(f(y))
	x : f(y)
1	assumption		{forall_{x in S | Q(x) , R(x)} P(x)} |- forall_{x in S | Q(x) , R(x)} P(x)
2	assumption		{f(y) in S} |- f(y) in S
3	assumption		{Q(f(y))} |- Q(f(y))
4	assumption		{R(f(y))} |- R(f(y))

This indicates that the proof requires a *specialization* step (step 0) and explicitly indicates, in the row under step 0, the mapping being performed (mapping $x$ to $f(y)$).  The subsequent proof steps that are required are simply proofs by assumption.  Specifically, the original `Forall` expression must be true and the conditions must be satisfied for the instance $x \mapsto f(y)$: $f(y) \in S$, $Q(f(x))$, and $R(f(y))$.  If any of these are not known to be true under the provided assumptions, this step will fail.  In this example, they are trivially true because our assumptions were chosen to be precisely what needed to be true for the *specialization* step to succeed.

If we leave out our first assumption (*i.e.*, the original `Forall` object itself), **Prove-It** is unable to prove the original `Forall` expression even after attempting to perform automation and the *specialization* step will fail.

In [44]:
from proveit import ProofFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[1:])
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove forall_{x in S | Q(x) , R(x)} P(x) assuming {f(y) in S, Q(f(y)), R(f(y))}: Unable to conclude automatically; the domain has no 'foldAsForall' method and automated generalization failed.


The automation checks to see if the $S$ **Expression** has a `foldAsForall` method that would automate a proof for universal quantification over $S$.  Since $S$ is a simple **Variable** object, no such automation exists.

Next we'll see what happens when the instantiated "instance" $f(y)$ is not known to be in the domain $S$ (*i.e.*, we omit the assumption that $f(y) \in S$):

In [45]:
from proveit import SpecializationFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:1]+assumptions[2:])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x) , R(x)} P(x), Q(f(y)), R(f(y))}: Unmet specialization requirement: f(y) in S


Finally, we demonstrate the case when one of the "explicit" conditions is not met -- first omitting the assumption that $Q(f(y))$ is TRUE:

In [46]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:2]+assumptions[3:])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x) , R(x)} P(x), f(y) in S, R(f(y))}: Unmet specialization requirement: Q(f(y))


then omitting the assumption that $R(f(y))$ is TRUE:

In [47]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:3])
    assert False, "Expecting a SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Unable to prove P(f(y)) assuming {forall_{x in S | Q(x) , R(x)} P(x), f(y) in S, Q(f(y))}: Unmet specialization requirement: R(f(y))


Also note that you cannot *specialize* a variable that is not one of the `Forall` instance variables. For example, we cannot specialize the propositional function variable $Q$. Recall our basic expression first:

In [48]:
basicForallExpr

forall_{x in S | Q(x) , R(x)} P(x)

Then try to *specialize* our `Forall` object by instantiating $Q$ with $R$:

In [49]:
try:
    basicForallExpr.specialize({x:fy, Q:R}, assumptions=assumptions)
    assert False, "Expecting a SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed assuming {forall_{x in S | Q(x) , R(x)} P(x), f(y) in S, Q(f(y)), R(f(y)), forall_{x in S | Q(x) , R(x)} P(x)}: May only specialize instance variables of directly nested Forall operations


You can, of course, *relabel* **Variable**s that are not `Forall` instance variables.  Later in this tutorial we will show that you can *relabel* and *specialize* simultaneously (see the sub-section on ***Specializing and Relabeling Simultaneously***).  You can also *specialize* multiple levels of `Forall` operations simultaneously which is why the previous error message mentions "nested Forall operations". 

### Universal quantification without a domain

It is not necessary to specify a domain in a `Forall` **Expression**.  For example, the condition(s) may provide sufficient restrictions for the universal quantification.  Also, any number of conditions may be specified (including no conditions).

In [50]:
noDomainForallExpr = Forall(x, Px, conditions=[Qx])

noDomainForallExpr: forall_{x | Q(x)} P(x)

We can verify the absence of an explicit domain in a variety of ways:

In [51]:
noDomainForallExpr.hasDomain()

False

In [52]:
assert noDomainForallExpr.domain is None  # The Forall object's domain is None

We can use the `exprInfo()` method to examine the internal structure of the expression:

In [53]:
noDomainForallExpr.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 8body: 3conditions: 4,
3,Operation,operator: 5operand: 8,
4,ExprList,6,
5,Variable,,
6,Operation,operator: 7operand: 8,
7,Variable,,
8,Variable,,


As before, we can implement a specialization or univeral instantiation, where the variable $x$ is replaced with the specific instance $f(y)$. To do so, we include the original `Forall` expression as an assumption, along with the assumption that $Q(f(y))$ is TRUE:

In [54]:
noDomainForallExpr.specialize({x:fy}, assumptions=[noDomainForallExpr, Function(Q, fy)])

{forall_{x | Q(x)} P(x) , Q(f(y))} |- P(f(y))

### Lambda scope restrictions

In `tutorial01_core_expr`, we noted scoping restrictions that apply to **Lambda** expressions in the context of *expression substitution*.  That restriction carries over to *specialization* and is very important.  Consider the following example.

In [55]:
from proveit.logic import NotEquals, Exists
from proveit._common_ import Pxy, y, fy
forallExistsExpr = Forall(x, Exists(y, NotEquals(x, y)))

forallExistsExpr: forall_{x} [exists_{y} (x != y)]

Note that, while `Forall` ($\forall$) has a special meaning in the **Prove-It** core, `Exists` ($\exists$) and `Equals` ($\neq$) do not (they are defined via **axioms** within the `proveit.logic` package, which we will explain in a later chapter).  We are using them here to make our point more clear.  Just note that `Exists` is another kind of `OperationOverInstances` that operates on a **lambda** function:

In [56]:
Exists(y, Pxy).exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 7body: 3,
3,Operation,operator: 4operands: 5,
4,Variable,,
5,ExprList,"6, 7",
6,Variable,,
7,Variable,,


If we try to specialize $x$ as $y$ in `nestedForall`, this will fail:

In [57]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:y}, assumptions={forallExistsExpr})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Must not make substitution with reserved variables  (i.e., parameters of a Lambda function)


And of course this *should* fail.  We cannot derive $\exists_y y \neq y$ by assuming $\forall_{ x } \left[\exists_y x \neq y \right]$.  The former is a stronger statement.  We chose this example, in fact, because the latter can be argued as typically true but the former is never true using reasonable definitions.  Where this goes wrong is in violating the scope of $\exists_y$.  It is introducing $y$ as a new **variable** within the sub-expression $\exists_y x \neq y$.  This **label** is off limits to $x$ which is quantified outside of this sub-expression.  We can *specialize* $x$ to whatever we want as long as we respect these scoping restrictions.

In fact, it is not simply $y$ that is off limits; all **expressions** involving $y$ are off limits. Assuming $\forall_{ x } \left[\exists_y x \neq y \right]$ should not allow us to derive $\exists_y f(y) \neq y$, and the attempted specialization does indeed fail:

In [58]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:fy}, assumptions={forallExistsExpr})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Must not make substitution with reserved variables  (i.e., parameters of a Lambda function)


It should also be noted that within a scope, a **variable** may be reused with a different meaning.  This should generally be avoided as it makes **expressions** unclear, but the functionality should be well-defined in case it ever happens.  If this happens, we treat it as a distinct **variable** from anything outside of the scope (that just happens to have the same name).  It can be confusing and should be avoided, but it is well-defined.  For example, consider the following nested `Forall` expression:

In [59]:
from proveit.logic import And
redundantInstanceVarExpr = Forall(x, And(Px, Forall(x, Qx)))

redundantInstanceVarExpr: forall_{x} (P(x) and [forall_{x} Q(x)])

Specializing the outer $x$ does not and should not change the inner $x$, which is treated as a distinct **variable**:

In [60]:
redundantInstanceVarExpr.specialize({x:fy}, assumptions={redundantInstanceVarExpr})

{forall_{x} (P(x) and [forall_{x} Q(x)])} |- P(f(y)) and [forall_{x} Q(x)]

In [61]:
redundantInstanceVarExpr.relabeled({x:y})

forall_{y} (P(y) and [forall_{y} Q(y)])

At this time, there is no straightforward way to specialize the inner $x$ itself (another reason to avoid such potentially confusing constructions).

### Operand, operator, or operation specialization

*Specializing* different parts of an **operation** works essentially the same way as it does with *expression substition*.

We will use the `substitution` axiom of `proveit.logic.equality` for demonstrations in this section out of convenience.  **Axioms** and the `proveit.logic` package will be discussed in more detail later.  For now, we note that **axioms** (and **theorems**) are taken to be true statements without proof, as you can see below.

In [62]:
from proveit.logic.equality._axioms_ import substitution
substitution

|- forall_{f, x, y | x = y} (f(x) = f(y))

In [63]:
substitution.proof()

step type	requirements	statement
0	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution

Let's grab the $x=y$ condition for use below.

In [64]:
x_eq_y = substitution.allConditions()[0]

x_eq_y: x = y

Operator and operand *specialization* are straightforward:

In [65]:
from proveit._common_ import f, g
operatorSubstitution = substitution.specialize({f:g})

operatorSubstitution: |- forall_{x, y | x = y} (g(x) = g(y))

In [66]:
operatorSubstitution.proof()

step type	requirements	statement
0	specialization	1	|- forall_{x, y | x = y} (g(x) = g(y))
	f : g
1	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution

Note that $x$ and $y$ mapped to themselves by default.  When a mapping is not specified, the default is to map the **variable** to itself.

Next we explicitly specialize the operands $x$ and $y$ to $a$ and $b$, respectively, in the original `substitution` axiom (and notice that we must explicitly include the assumption that $a = b$, else the `substitution` axiom will not apply):

In [67]:
from proveit._common_ import a, b
from proveit.logic import Equals
a_eq_b = Equals(a, b)
operandSubstitution = substitution.specialize({x:a, y:b}, assumptions=[a_eq_b])

operandSubstitution: {a = b} |- f(a) = f(b)

In [68]:
operandSubstitution.proof()

step type	requirements	statement
0	specialization	1, 2	{a = b} |- f(a) = f(b)
	f : f, x : a, y : b
1	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
2	assumption		{a = b} |- a = b

Operation substitution can be done explicitly via a **lambda** expression just as we saw with *expression substitution*.  The **lambda** expression is not literally substituted in; rather, the function that it represents is applied as the operation.

In [69]:
operationSubstitution = substitution.specialize({f:Lambda(x, Add(x, a))})

operationSubstitution: |- forall_{x, y | x = y} ((x + a) = (y + a))

In [70]:
operationSubstitution.proof()

step type	requirements	statement
0	specialization	1	|- forall_{x, y | x = y} ((x + a) = (y + a))
	f(x) : x + a
1	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution

An alternative way to specify an operation substitution is to map the operation applied to a **variable** (*e.g.*, the operation $f$ applied to $x$ as $f(x)$) onto an **expression** that uses this same **variable** (*e.g.*, `Add(x, a)`).  In the example below, we map $f(x)$ to $x + a$.  This will be internally translated to the same **lambda** expression as before: $x \mapsto x + a$.

In [71]:
from proveit._common_ import fx
operationSubstitution2 = substitution.specialize({fx:Add(x, a)})

operationSubstitution2: |- forall_{x, y | x = y} ((x + a) = (y + a))

The proof is exactly the same as before.

In [72]:
operationSubstitution2.proof()

step type	requirements	statement
0	specialization	1	|- forall_{x, y | x = y} ((x + a) = (y + a))
	f(x) : x + a
1	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution

### Specializing multiple levels simultaneously

When `Forall` operations are nested, the universal quantifications may be specialized separately.

Consider, for example, the nested quantification expression $\forall_x [\forall_y [\forall_{z|z<(x+y)} P(x, y, z)]]$, defined below:

In [73]:
from proveit._common_ import z, Pxyz
from proveit.number import Less, Add
nestedForall = Forall(x, Forall(y, Forall(z, Pxyz, conditions=[Less(z, Add(x, y))])))

nestedForall: forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]

We can successively specialize to reach the inner-most instance-defining variable $z$:

In [74]:
nestedForallSpec1 = nestedForall.specialize(assumptions=[nestedForall])

nestedForallSpec1: {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]

In [75]:
nestedForallSpec2 = nestedForallSpec1.specialize()

nestedForallSpec2: {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{z | z < (x + y)} P(x , y , z)

In [76]:
nestedForallSpec3 = nestedForallSpec2.specialize(assumptions=[nestedForallSpec2.conditions[0]])

nestedForallSpec3: {z < (x + y) , forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- P(x , y , z)

In [77]:
nestedForallSpec3.proof()

step type	requirements	statement
0	specialization	1, 2	{z < (x + y) , forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- P(x , y , z)
	z : z
1	specialization	3	{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{z | z < (x + y)} P(x , y , z)
	y : y
2	assumption		{z < (x + y)} |- z < (x + y)
3	specialization	4	{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]
	x : x
4	assumption		{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]

But for the sake of convenience and efficiency, **Prove-It** also allows nested `Forall` operations to be specialized simultaneously:

In [78]:
assumptions = ExprList(nestedForall, nestedForallSpec2.conditions[0])
nestedForallSimultaneousSpec = nestedForall.specialize({z:z}, assumptions=assumptions)

nestedForallSimultaneousSpec: {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]] , z < (x + y)} |- P(x , y , z)

We just need to include an explicit mapping for an inner quantified **variable**.  So we indicated above that we want to map $z$ to $z$, even though this is typically the default, in order to force it to specialize all three `Forall` operations simultaneously.  The proof is shorter, doing a single all-in-one *specialization*:

In [79]:
nestedForallSimultaneousSpec.proof()

step type	requirements	statement
0	specialization	1, 2	{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]] , z < (x + y)} |- P(x , y , z)
	x : x, y : y, z : z
1	assumption		{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]
2	assumption		{z < (x + y)} |- z < (x + y)

Such simultaneous *specialization* can be done for any number of nested levels.

As a quick aside, it worth noting that the original nested `Forall` involving the instance variables $x$, $y$, and $z$ could have instead been expressed as $\forall_{x, y, z | z < x + y} P(x, y, z)$, constructed in the following way (also see the section further below on “Universal quantification over multiple variables”):

In [80]:
altNestedForall = Forall([x, y, z], Pxyz, conditions=[Less(z, Add(x, y))])

altNestedForall: forall_{x, y, z | z < (x + y)} P(x , y , z)

Note that the *meaning* of the original **nestedForall** object $\forall_x [\forall_y [\forall_{z|z<x+y} P(x, y, z)]]$ and the meaning of our alternative **altNestedForall** object $\forall_{x, y, z | z < x + y} P(x, y, z)$ are the same:

In [81]:
altNestedForall == nestedForall

True

We can now accomplish the same specialization as follows:

In [82]:
altNestedForallSpec = altNestedForall.specialize(assumptions=[altNestedForall, Less(z, Add(x, y))])

altNestedForallSpec: {forall_{x, y, z | z < (x + y)} P(x , y , z) , z < (x + y)} |- P(x , y , z)

with similar proof details:

In [83]:
altNestedForallSpec.proof()

step type	requirements	statement
0	specialization	1, 2	{forall_{x, y, z | z < (x + y)} P(x , y , z) , z < (x + y)} |- P(x , y , z)
	x : x, y : y, z : z
1	assumption		{forall_{x, y, z | z < (x + y)} P(x , y , z)} |- forall_{x, y, z | z < (x + y)} P(x , y , z)
2	assumption		{z < (x + y)} |- z < (x + y)

### Specializing and relabeling simultaneously

It is also possible to achieve *relabeling* and *specialization* (over any number of nested levels) in a single step.

For example, recall our nested `Forall` object $\forall_x [\forall_y [\forall_{z|z<x+y} P(x, y, z)]]$ from above (explicitly redefined here in case changes were made above): 

In [84]:
nestedForall = Forall(x, Forall(y, Forall(z, Pxyz, conditions=[Less(z, Add(x, y))])))

nestedForall: forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]

We can specialize the outermost instance variable $x$ while simultaneously relabeling the instance variable $z$ to $a$:

In [85]:
nestedForall.specialize(relabelMap={z:a}, assumptions=[nestedForall])

{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{y} [forall_{a | a < (x + y)} P(x , y , a)]

Or we can specialize all the way down to the innermost instance variable ($z$) while simultaneously relabeling the instance variable $z$ to $a$:

In [86]:
nestedForallSpecAndRelab = nestedForall.specialize(specializeMap={y:y}, relabelMap={z:a}, assumptions=[nestedForall])

nestedForallSpecAndRelab: {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{a | a < (x + y)} P(x , y , a)

In that `specialize()` command, we used the explicit `specializeMap` and `relabelMap` designations to distinguish the two types of mappings, but we could have relied simply on the order of the input arguments instead, with the specialization mapping(s) expected first and the relabel mapping(s) expected second:

In [87]:
nestedForall.specialize({y:y}, {z:a}, assumptions=[nestedForall, Less(a, Add(x, y))])

{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{a | a < (x + y)} P(x , y , a)

We can take a quick look at the related proof to see how the specialization and relabeling mappings are indicated:

In [88]:
nestedForallSpecAndRelab.proof()

step type	requirements	statement
0	specialization	1	{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{a | a < (x + y)} P(x , y , a)
	x : x, y : y, relabeling z : a
1	assumption		{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]

The *relabeling* map is always shown after the last comma in the mapping under the *specialization* step.  When any *specialization* occurs, the "step type" is labeled "specialization".  If there is only *relabeling*, the "step type" will indicate "relabeling". 

You are not allowed to specify that the same **variable** is to be *specialized* and *relabeled*. For example, recall once again our nested `Forall` object:

In [89]:
nestedForall

forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]

We can specialize the inner instance variable $y$ to $b$:

In [90]:
nestedForall.specialize({y:b}, assumptions=[nestedForall])

{forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]} |- forall_{z | z < (x + b)} P(x , b , z)

But we cannot simultaneously specialize $y$ to $b$ *and also* relabel $y$ with $a$:

In [91]:
try:
    nestedForall.specialize({y:b}, {y:a}, assumptions=[nestedForall])
    assert False, "Expecting a SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed assuming {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]}: Attempting to specialize and relabel the same variable: y


As noted in the previous tutorial chapter (tutorial04_relabeling), relabeling has another important limitation.  You cannot relabel something using assumptions that involve any of the relabeling variables.  For example, we cannot relabel $P$ to $R$ in `nestedForall` while assuming `nestedForall`. (Would be nice to elaborate on this briefly.)

In [92]:
from proveit import RelabelingFailure
try:
    nestedForall.specialize({y:y}, {P:R}, assumptions=[nestedForall])
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Proof step failed assuming {forall_{x} [forall_{y} [forall_{z | z < (x + y)} P(x , y , z)]]}: Attempting to relabel variable(s) that are free in the assumptions: {P}


### Universal quantification over multiple variables

Rather than nesting `Forall` operations, you can quantify over multiple instance variables for a more succinct expression. For example, instead of writing $\forall_{x\,\in\ S} [\forall_{y\,\in\, S} P(x, y)]$, you could write $\forall_{x, y\,\in\, S} P(x, y)$, constructed like this:

In [93]:
multiVarForall = Forall((x, y), Pxy, domain=S)

multiVarForall: forall_{x, y in S} P(x , y)

Specialization is still achieved in the same way. Here, for example, we assume the `Forall` expression along with specific $x$ and $y$ values in set $S$:

In [94]:
assumptions = [multiVarForall, InSet(x, S), InSet(y, S)]
multiVarForallSpec = multiVarForall.specialize(assumptions=assumptions)

multiVarForallSpec: {forall_{x, y in S} P(x , y) , x in S , y in S} |- P(x , y)

We can see how **ProveIt** conceptualizes that specialization by calling its `proof()` method:

In [95]:
multiVarForallSpec.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{forall_{x, y in S} P(x , y) , x in S , y in S} |- P(x , y)
	x : x, y : y
1	assumption		{forall_{x, y in S} P(x , y)} |- forall_{x, y in S} P(x , y)
2	assumption		{x in S} |- x in S
3	assumption		{y in S} |- y in S

Note that when constructing a `Forall` object, if you attempt to use the same **variable** multiple times in the list of instance variables, you will get an error:

In [96]:
try:
    Forall((x, x), Px)
except ValueError as e:
    print('EXPECTED ERROR:', e)

You can also specify different domains for each a set of instance **variables**, providing a list (or `ExprList`) for `domains` rather than `domain`.  The display notation will indicate a cartesian product set.

In [97]:
multiDomainForall = Forall((x, y), Pxy, domains=[S, R])

multiDomainForall: forall_{(x, y) in S * R} P(x , y)

Internally, however, **ProveIt** simply splits off a condition for each *instance variable*. Notice lines 6 and 12 in the following `exprInfo()` details:

In [98]:
multiDomainForall.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 4operand: 1,
1,Lambda,parameter: 14body: 2conditions: 3,
2,Operation,operator: 4operand: 5,
3,ExprList,6,
4,Literal,,
5,Lambda,parameter: 17body: 7conditions: 8,
6,Operation,operator: 15operands: 9,
7,Operation,operator: 10operands: 11,
8,ExprList,12,
9,ExprList,"14, 13",


We can perform a specialization of $\forall_{(x, y)\,\in\, S \,\times\, R} P(x, y)$ analogous to the previous specialization of $\forall_{x, y\,\in\, S} P(x, y)$, but now specifying distinct set-inclusion assumptions for the variables $x$ and $y$:

In [99]:
assumptions = [multiDomainForall, InSet(x, S), InSet(y, R)]
multiDomainForallSpec = multiDomainForall.specialize(assumptions=assumptions)

multiDomainForallSpec: {forall_{(x, y) in S * R} P(x , y) , x in S , y in R} |- P(x , y)

In [100]:
multiDomainForallSpec.proof()

step type	requirements	statement
0	specialization	1, 2, 3	{forall_{(x, y) in S * R} P(x , y) , x in S , y in R} |- P(x , y)
	x : x, y : y
1	assumption		{forall_{(x, y) in S * R} P(x , y)} |- forall_{(x, y) in S * R} P(x , y)
2	assumption		{x in S} |- x in S
3	assumption		{y in R} |- y in R

Universal quantification over an unspecified number of **variables** via **iterations** will be discussed in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>.

Generalization
========

*Generalization* is the "inverse" of *specialization* just as *hypothetical reasoning* was the "inverse" of *modus ponens*.  We can write these derivation rules in a manner that makes this relationship clear (just as we did for *hypothetical reasoning* and *modus ponens*):

Specialization: 
$\begin{array}{c}
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x) \\
\hline
\left \{\clubsuit \in S,~Q(\clubsuit) \right \} \boldsymbol{\vdash} P(\clubsuit)
\end{array}$

Generalization: 
$\begin{array}{c}
\left \{ x \in S,~Q(x) \right \} \boldsymbol{\vdash} P(x) \\
\hline
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x)
\end{array}$

$P(x)$ and $Q(x)$ are intended to represent any function of $x$.
There is some asymmetry between *specialization* and *generalization*.  $\clubsuit$ here is meant to represent *any* **expression**, not necessarily a **variable**, as long is it does not violate scoping restrictions (*e.g.*, having a free **variable** that is the same as a **lambda** *parameter* within the $P$ or $Q$ functions).  However, *generalization* only applies to an unbound **variable**.  In Prove-It, an unbound **variable** is regarded as an "arbitrary" variable.  Essentially, it is implicitly universally quantified.  Recall that *modus ponens* converts an explicit antecedent to an implicit assumption and *hypothetical reasoning* does the opposite.  Similarly, *specialization* converts an explicit universal quantification to implicit arbitrary variables and *generalization* does the opposite.  The reason for having the explicit and implicit forms is much the same as it was for the antecedent versus assumption.  The explicit form allows nesting but the implicit form provides direct access to the instance expression.  Furthermore, explicit universal quantification offers the power of being able to *specialize* an instance variable to an arbitrary **expression**.

The above derivation rules are expressed for a single **variable**.  Such rules apply more generally to any number of **variables** (including an unspecified number of **variables** via **iterations** discussed in in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>).

The following examples will start from a *specialization* instance of the `substitution` **known truth** visited earlier:

In [101]:
substitution

|- forall_{f, x, y | x = y} (f(x) = f(y))

In [102]:
operationSubstitution = substitution.specialize({f:Lambda(x, Add(x, a)), x:x, y:y}, assumptions=[Equals(x, y)])

operationSubstitution: {x = y} |- (x + a) = (y + a)

First, we try to *generalize* this **known truth** for all instances of $a$, $x$, and $y$ without any conditions or domain restrictions:

In [103]:
from proveit import GeneralizationFailure
try:
    operationSubstitution.generalize((a, x, y))
    assert False, "Expecting a GeneralizationFailure error; should not make it to this point"
except GeneralizationFailure as e:
    print('EXPECTED ERROR:', e)

EXPECTED ERROR: Unable to prove forall_{a, x, y} ((x + a) = (y + a)) assuming {x = y}: Cannot generalize using assumptions that involve any of the new forall variables (except as assumptions are eliminated via conditions or domains)


That `generalize()` attempt fails because the assumptions of the original **known truth** involve the same **variables** that we are trying to *generalize* over.  That is not allowed because universal quantification introduces a new scope for $x$ and $y$ (as well as $a$) and the $x=y$ assumption would be external to this scope.  If, however, this assumption is introduced as a condition of the new universal quantification, then we no longer need to retain it as an assumption.  That assumption will be absorbed into the universal quantification conditions.

In [104]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y])

|- forall_{a, x, y | x = y} ((x + a) = (y + a))

Adding additional restrictions, such as a domain and/or other conditions, only makes the statement weaker and is therefore allowed:

In [105]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y], domain=S)

|- forall_{a, x, y in S | x = y} ((x + a) = (y + a))

In [106]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y, Qx], domain=S)

|- forall_{a, x, y in S | x = y , Q(x)} ((x + a) = (y + a))

It is also possible to create multiple levels of nested `Forall` operations in one step with possibly different domains.  Simply provide a list of lists of **Variable**s as the first argument to `generalize` and a corresponding list of `domains`.  The conditions are applied at the outermost level possible (as soon as all of the relevant variables have been introduced) but otherwise retaining the order that the `conditions` were supplied.

In [107]:
Qa = Function(Q, a)
nestedGenExample = operationSubstitution.generalize([[a], [x, y]], domainLists=[[P], [R, S]], conditions=[x_eq_y, Qx, Qa])

nestedGenExample: |- forall_{a in P | Q(a)} [forall_{(x, y) in R * S | x = y , Q(x)} ((x + a) = (y + a))]

Note that the $Q(a)$ condition was moved to the front even though it was the last supplied condition because it can be applied before the others.  The *generalization* with multiple leves of nested `Forall` operations takes one step in the proof:

In [108]:
nestedGenExample.proof()

step type	requirements	statement
0	generalizaton	1	|- forall_{a in P | Q(a)} [forall_{(x, y) in R * S | x = y , Q(x)} ((x + a) = (y + a))]
1	specialization	2, 3	{x = y} |- (x + a) = (y + a)
	f(x) : x + a, x : x, y : y
2	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
3	assumption		{x = y} |- x = y

For any level in which multiple instance variables are introduced, a domain can be supplied for each new variable (as above) or one may be specified as the same domain for each of them:

In [109]:
operationSubstitution.generalize([[a], [x, y]], domainLists=[[P], [R]], conditions=[x_eq_y, Qx, Qa])

|- forall_{a in P | Q(a)} [forall_{x, y in R | x = y , Q(x)} ((x + a) = (y + a))]

When *generalizing* over a single **variable**, the first argument may be just that **variable** rather than a list or tuple:

In [110]:
operationSubstitution.generalize(a, conditions=[Qa])

{x = y} |- forall_{a | Q(a)} ((x + a) = (y + a))

We may only *generalize* over **variables** (or **iterations** of **variables**) however:

In [111]:
try:
    operationSubstitution.generalize(Qx, conditions=[Qa])
    assert False, "Expecting an ValueError error; should not make it to this point"    
except ValueError as e:
    print('EXPECTED ERROR:', e)

EXPECTED ERROR: Must supply 'generalize' with a Variable, list of Variables, or list of Variable lists.


In [112]:
try:
    operationSubstitution.generalize([[a], [Qx]], conditions=[Qa])
    assert False, "Expecting an ValueError error; should not make it to this point"
except ValueError as e:
    print('EXPECTED ERROR:', e)

EXPECTED ERROR: Forall variables of a generalization must be Variable objects


In [113]:
%end skolemization_explorations

*Generalizing* an unspecified number of **variables** via an **iteration** will be discussed in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>.